In [18]:
# External modules
import torch
import tqdm
import time

import numpy as np
import matplotlib.pyplot as plt

from torchvision import transforms
from torch.utils.data import DataLoader, random_split, Subset
from torchvision.models import resnet18
from tqdm.notebook import tqdm as tqdm

from sklearn.model_selection import train_test_split

import sys
import os

project_root = os.path.abspath("..")
sys.path.append(project_root)

# owned modules
from src.datasets import SCINDataset
from src.models import custom_collate_fn, fgsm_attack, denorm, test_fgsm_attack

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(42)
np.random.seed(42)

ImportError: cannot import name 'test_fgsm_attack' from 'src.models' (/Users/renad/Documents/code/fairness-embeddings/src/models.py)

# Loading dataset

In [2]:
resnet18_mean = [0.485, 0.456, 0.406]
resnet18_std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=resnet18_mean,
                         std=resnet18_std)          
])

dataset = SCINDataset(
    root_dir="../data/external/scin/dataset",
    labels_csv="scin_labels.csv",
    cases_csv="scin_cases.csv",
    transform=transform
)

num_classes = len(dataset.label_encoder.classes_)

labels = dataset.data['weighted_skin_condition_label'].values
indices = list(range(len(dataset)))

train_indices, val_indices = train_test_split(indices, test_size=0.2, shuffle=True, random_state=42)

train_data = Subset(dataset, train_indices)
val_data = Subset(dataset, val_indices)

train_loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=custom_collate_fn)
val_loader = DataLoader(val_data, batch_size=32, shuffle=True, collate_fn=custom_collate_fn)


In [3]:
print("Number of training samples:", len(train_data))
print("Number of validation samples:", len(val_data))
print(len(dataset))

Number of training samples: 4025
Number of validation samples: 1007
5032


# Resnet18

In [4]:
# model
batch_size = 16
lr = 0.1
num_epochs = 1

# adversarial
epsilon = 0.05 # 0, .05, .1, .15, .2, .25, .3

In [5]:
model = resnet18(pretrained=True).to(device)

model.fc = torch.nn.Linear(512, num_classes)

for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=lr)

/Users/renad/Documents/code/fairness-embeddings/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/renad/Documents/code/fairness-embeddings/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
train_accuracy = []
val_accuracy = []

for epoch in tqdm(range(num_epochs)):
    
    start_time = time.time()
    
    # TRAINING
    running_loss = 0
    correct = 0
    total = 0
    
    train_epoch_time = time.time()
    model.train()
    for batch in tqdm(train_loader, desc=f"Training Epoch [{epoch+1}/{num_epochs}]"):
        data, target = batch
        data, target = data.to(device), target.to(device)

        model.zero_grad()
        data.requires_grad = True
        outputs = model(data)
        loss = loss_fn(outputs, target)
        loss.backward()
        
        data_grad = data.grad.data
        perturbed_data = fgsm_attack(data, epsilon, data_grad)
        
        combined_data = torch.cat((data, perturbed_data), dim=0)
        combined_target = torch.cat((target, target), dim=0)
        
        optimizer.zero_grad()
        outputs = model(combined_data)
        loss = loss_fn(outputs, combined_target)
        loss.backward()
        optimizer.step()
        
        predicted = torch.argmax(outputs[:data.size(0)], 1) 
        total += target.size(0)
        correct += (predicted == target).sum().item()
        
        running_loss += loss.item()

        epoch_accuracy = 100 * correct / total
        train_accuracy.append(epoch_accuracy)

    train_time = time.time() - train_epoch_time
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')


    # VALIDATION    
    val_correct = 0
    val_total = 0
    
    val_epoch_time = time.time()
    model.eval()
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Validation Epoch [{epoch+1}/{num_epochs}]"):
            data, target = batch
            data, target = data.to(device), target.to(device)
            
            outputs = model(data)
            predicted = torch.argmax(outputs, 1)
            val_total += target.size(0)
            val_correct += (predicted == target).sum().item()
            
        val_epoch_accuracy = 100 * val_correct / val_total
        val_accuracy.append(val_epoch_accuracy)
    
    val_time = time.time() - val_epoch_time    
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {val_epoch_accuracy:.2f}%')
        
    epoch_time = time.time() - start_time
    print(f"Epoch [{epoch+1}/{num_epochs}] completed in {epoch_time:.2f}s (Train: {train_time:.2f}s, Val: {val_time:.2f}s)")
    

  0%|          | 0/1 [00:00<?, ?it/s]

Training Epoch [1/1]:   0%|          | 0/158 [00:00<?, ?it/s]

Epoch [1/1], Loss: 40715.4126, Accuracy: 15.42%


Validation Epoch [1/1]:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch [1/1], Validation Accuracy: 1.59%
Epoch [1/1] completed in 1034.14s (Train: 980.89s, Val: 53.25s)


In [ ]:
fig = plt.figure(figsize=(14,8))
plt.plot(range(1, num_epochs+1,1), train_accuracy, label="Training accuracy")
plt.plot(range(1, num_epochs+1,1), val_accuracy, label="Validation accuracy")
plt.xticks(range(1, num_epochs + 1))
plt.suptitle("Model accuracy across epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.show()

In [17]:
acc, ex = test_fgsm_attack(model, device, val_loader, epsilon)
print(f"Accuracy on adversarial examples: {acc:.2f}%")
print(f"Number of examples: {ex}")

NameError: name 'test_fgsm_attack' is not defined